# Inteligent Rock-Paper-Scissors Game!
### 1- Importing Dependensies & Libraries

In [2]:
import re
import os
import sys
import time
import uuid
import torch
import random
import shutil
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook

from ultralytics import YOLO

print("Python Version:", sys.version)
"""Consider to create a venv with python 3.10 or lower to prevent facing any issues with the libraries"""

Python Version: 3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]


'Consider to create a venv with python 3.10 or lower to prevent facing any issues with the libraries'

##### Defining the pathes of images and labels

In [3]:
# os.mkdir("Dataset/")

# os.mkdir("Dataset/Images/")
# os.mkdir("Dataset/Labels/")
# os.mkdir("Dataset/Texts/")

# os.mkdir("Dataset/Images/Rock/")
# os.mkdir("Dataset/Images/Paper/")
# os.mkdir("Dataset/Images/Scissors/")

# os.mkdir("Dataset/Labels/Rock/")
# os.mkdir("Dataset/Labels/Paper/")
# os.mkdir("Dataset/Labels/Scissors/")

# os.mkdir("Dataset/Texts/Rock/")
# os.mkdir("Dataset/Texts/Paper/")
# os.mkdir("Dataset/Texts/Scissors/")

images_path = os.path.join("Dataset/Images/")
labels_path = os.path.join("Dataset/Lables/")

rock_images_path = os.path.join("Dataset/Images/Rock/")
paper_images_path = os.path.join("Dataset/Images/Paper/")
scissors_images_path = os.path.join("Dataset/Images/Scissors/")

rock_labels_path = os.path.join("Dataset/Labels/Rock/")
paper_labels_path = os.path.join("Dataset/Labels/Paper/")
scissors_labels_path = os.path.join("Dataset/Labels/Scissors/")

classes = [{'Name':'Rock', 'id':0}, {'Name':'Paper', 'id':1}, {'Name':'Scissors', 'id':2}]

### 2- Capturing Images for dataset 
###### (No any published dataset will be used - ONLY GATHERED DATASET)

In [ ]:
cam = cv.VideoCapture(0)
 
# Change the path for each class everytime
path = rock_labels_path

# Capture 35 images for each class
count = 1
while cam.isOpened():
    ret, frame = cam.read()
    img_path = os.path.join(path, f'{str(uuid.uuid1())}.jpg')
    cv.imshow('Camera', frame)
    time.sleep(0.2)
    if cv.waitKey(1) & 0xFF == ord('c'):
        cv.imwrite(img_path, frame)
        print(f'Image "{count}" captured')
        count += 1
    elif cv.waitKey(1) & 0xFF == ord('q'):
        print('Closing webcam...')
        break

    if count == 36:
        print('\n35 Images Collected\nClosing webcam...')
        break
    
cam.release()
cv.destroyAllWindows()

### 3- Use LabelImg to make labels for each image
* pip install labelImg
* open cmd
* type labelImg
##### Convert xml files coordinations into text file
* arrangement:
    * 0. label
    * 1. xmin
    * 2. ymin
    * 3. xmax
    * 4. ymax 

In [ ]:
"""Also there are some libraries like 'xmltodict' for converting .xml datas into .txt format"""
# pip install xmltodict

def xml_to_txt(label_xml, label_path, label_txt, label_id):
    for i in range(len(label_xml)):
        with open(f'{label_path}{label_xml[i]}') as f:
            file = f.read()
            file = file.splitlines()

        xmin = int(re.split("\t|</xmin>|<xmin>", file[19])[-2])
        ymin = int(re.split("\t|</ymin>|<ymin>", file[20])[-2])
        xmax = int(re.split("\t|</xmax>|<xmax>", file[21])[-2])
        ymax = int(re.split("\t|</ymax>|<ymax>", file[22])[-2])
        
        with open(f"{label_txt}{label_xml[i].split('.xml')[0]}.txt", 'w') as f:
            
            # Change the below line code format in case you wanted to use another model
            f.write(f"{label_id} {xmin} {ymin} {xmax} {ymax}")
    
    return(print("Operation was successful!"))

# Reminder: classes = [{'Name':'Rock', 'id':0}, {'Name':'Paper', 'id':1}, {'Name':'Scissors', 'id':2}]
rock_xml = os.listdir(rock_labels_path)
paper_xml = os.listdir(paper_labels_path)
scissors_xml = os.listdir(scissors_labels_path)

rock_txt_path = os.path.join("Dataset/Texts/Rock/")
paper_txt_path = os.path.join("Dataset/Texts/Paper/")
scissors_txt_path = os.path.join("Dataset/Texts/Scissors/")

xml_to_txt(rock_xml, rock_labels_path, rock_txt_path, 0)
xml_to_txt(paper_xml, paper_labels_path, paper_txt_path, 1)
xml_to_txt(scissors_xml, scissors_labels_path, scissors_txt_path, 2)

In [ ]:
def rectangle_drawer(file_name, label_path, image_path):
    with open(f"{label_path}{file_name}.txt") as f:
        coords = f.read().split()
    
    image = cv.imread(f"{image_path}{file_name}.jpg")
    
    # Extract top-left and bottom-right points
    top_left = (int(coords[1]), int(coords[2]))
    bottom_right = (int(coords[3]), int(coords[4]))

    # Draw rectangle on the image
    image = cv.rectangle(image, top_left, bottom_right, (0, 255, 0), 5)
    return(image)

rock_test_img = rectangle_drawer("34d591ed-6516-11ef-9f12-a289da6cb5d8", rock_txt_path, rock_images_path)
paper_test_img = rectangle_drawer("5356c03a-6515-11ef-a75b-a289da6cb5d8", paper_txt_path, paper_images_path)
scissors_test_img = rectangle_drawer("03d68a77-6516-11ef-9436-a289da6cb5d8", scissors_txt_path, scissors_images_path)

### 4- Testing the boundary boxes that we've created early using "LabelImg"

In [ ]:
plt.figure(figsize=(16, 8))

plt.subplot(131)
plt.imshow(cv.cvtColor(rock_test_img, cv.COLOR_BGR2RGB))
plt.title("Rock", c='green')

plt.subplot(132)
plt.imshow(cv.cvtColor(paper_test_img, cv.COLOR_BGR2RGB))
plt.title("Paper", c='green')

plt.subplot(133)
plt.imshow(cv.cvtColor(scissors_test_img, cv.COLOR_BGR2RGB))
plt.title("Scissors", c='green')

plt.show()

Since the standard Yolo-Annotation format includes a ".txt" file which has:
#### * class_name -> x-center -> y-center -> width -> height
we have to change the values of text files

### Caution: label values should become normalized to use for "YOLO"


# RUN THE BELOW CELL ONLY ONCE!

In [ ]:
# All images are in the same shape
image = cv.imread("Dataset/Images/Rock/32cb2d4f-6516-11ef-b2d1-a289da6cb5d8.jpg")
h_img, w_img, channel = image.shape

text_folder = os.listdir(os.path.join("Dataset/Texts/"))

for i in range(len(text_folder)):
    text_files = os.listdir(os.path.join(f"Dataset/Texts/{text_folder[i]}"))
    for j in range(len(text_files)):
        with open(f"Dataset/Texts/{text_folder[i]}/{text_files[j]}") as f:
            # Label - xmin - ymin - xmax - ymax
            file = list(map(int, f.read().split()))
        
        label = file[0]
        w_box = (file[3] - file[1])
        h_box = (file[4] - file[2])
        x_center = (file[1] + w_box / 2) / w_img
        y_center = (file[2] + h_box / 2) / h_img
        w_box = w_box / w_img
        h_box = h_box / h_img
        with open(f"Dataset/Texts/{text_folder[i]}/{text_files[j]}", 'w') as f:
            f.write(f"{label} {x_center} {y_center} {w_box} {h_box}")

#### How to build "Yolo-setup.yaml" file:
1. (Optional): Create a folder for the config and train weights
2. Create "Yolo-setup.yaml" file
3. Define path, train, val & names for your problem
4. Put the 'path' of where your python code and dataset is
5. Put the path of your images in train and val

In [ ]:
# Now cather all datas in a single folder

# os.mkdir("Yolo-Data/")
# os.mkdir("Yolo-Data/train/")
# os.mkdir("Yolo-Data/val/")
# os.mkdir("Yolo-Data/train/images")
# os.mkdir("Yolo-Data/train/labels")
# os.mkdir("Yolo-Data/val/images")
# os.mkdir("Yolo-Data/val/labels")

def yolo_annotator(folder_path, file_type=0):
    for i in tqdm_notebook(range(len(os.listdir(folder_path)))):
        for file in os.listdir(folder_path):
            if file_type == 0 or file_type == 'image':
                shutil.copy(f"{folder_path}{file}", "Yolo-Data/train/images")
                
            elif file_type == 1 or file_type == 'label':
                shutil.copy(f"{folder_path}{file}", "Yolo-Data/train/labels")
    
yolo_annotator(rock_images_path, 0)
yolo_annotator(paper_images_path, 0)
yolo_annotator(scissors_images_path, 0)

yolo_annotator(rock_txt_path, 1)
yolo_annotator(paper_txt_path, 1)
yolo_annotator(scissors_txt_path, 1)

for _ in range(20):
    random_img = random.choice(os.listdir(os.path.join("Yolo-Data/train/images/")))
    shutil.move(f"Yolo-Data/train/images/{random_img}", f"Yolo-Data/val/images/{random_img}")
    random_lbl = random_img.split(".jpg")[0]+'.txt'
    shutil.move(f"Yolo-Data/train/labels/{random_lbl}", f"Yolo-Data/val/labels/{random_lbl}")

### Training Model
* I recommend you to train the model with more datasets and epochs to get better result

In [ ]:
# Configuring .yaml file for the pathes of each class for training the YOLO model
with open("setting.yaml", 'w') as f:
    f.write("""
path: /Users/Sina/Desktop/Summer Projects/Rock Paper Scissors/Yolo-Data
train: train/images
val: val/images

nc: 3
names:
    0: Rock
    1: Paper
    2: Scissors
""")

# It's possible to use torch to load YOLO model but i recommend you to use it the same way i did
model = YOLO("yolov5s.pt")
results = model.train(data="setting.yaml", epochs=50, imgsz=640)